In [ ]:
import torch
import matplotlib.pyplot as plt
import glob
import json
import os
import re

In [ ]:
torch.set_printoptions(precision=10)
PATH = "."
DATA_DIR = os.path.join(PATH, 'data')

In [ ]:
%run models/rendernet.py
%run data_loaders/scannet_render_loader.py

In [ ]:
# Load a trained model using its best weights unless otherwise specified
def load_trained_model(train_id, checkpoint_name='model_best'):
    model_path = os.path.join(PATH, 'saved/models/DNR', train_id)

    # Load config file
    config_file = os.path.join(model_path, "config.json")
    if config_file:
        with open(config_file, 'r') as f:
            config = json.load(f)

    # Load model weights
    checkpoint_path = os.path.join(model_path, checkpoint_name) + '.pth'
    
    if not os.path.isfile(checkpoint_path):
        pose_files = glob.glob(os.path.join(model_path, '*.pth'))
        
        def extract_epoch(f):
            s = re.findall("checkpoint-epoch(\d+)", f)
            return (int(s[0]) if s else -1,f)
        
        checkpoint_path = max(pose_files,key=extract_epoch)
    
    print('loaded:', checkpoint_path)
    
    checkpoint = torch.load(checkpoint_path, map_location='cpu')

    # Handling loading of models from different versions will be tricky.
    # Eventually might need to check models out from git repo. For now,
    # use these fixes.
    if 'mipmap_levels' in config['arch']['args']:
        mipmap_levels = config['arch']['args']['mipmap_levels']
        zero_other_mipmaps = False
        print('contains mipmap:', mipmap_levels)
    else:
        mipmap_levels = 1
        checkpoint['state_dict']['neural_texture.mipmap.0'] = checkpoint['state_dict'].pop('neural_texture.texture')
        zero_other_mipmaps = True
        print('Warning: {} weights from old model version with only one mipmap layer'.format(train_id))
    
    # Ahh! We have to hack the weights since ParameterList isn't accepted by TorchScript
    if 'neural_texture.mipmap.0' in checkpoint['state_dict']:
        print('Warning: {} mipmap from old model version that used ParameterList or a single layer'.format(train_id))
        for i in range(mipmap_levels):
            checkpoint['state_dict']['neural_texture.mipmap_{}'.format(i)] = checkpoint['state_dict'].pop('neural_texture.mipmap.{}'.format(i))
    
    # Load model with parameters from config file
    model = RenderNet(config['arch']['args']['texture_size'],
                     config['arch']['args']['texture_depth'],
                     mipmap_levels)
    
    # TODO: WARNING: Leaving some mipmap layer weights unassigned might lead to erroneous
    #  results (maybe they're not set to zero by default)
    # Assign model weights and set to eval (not train) mode
    model.load_state_dict(checkpoint['state_dict'], strict=(not zero_other_mipmaps))
    model.eval()
    
    return model

In [ ]:
# load all models by train id
def load_trained_models(train_ids):
    models = {}
    for train_id in train_ids:
        models[train_id] = load_trained_model(train_id)
    
    return models

In [ ]:
# Cereate a libtorch script file containing the model that can be loaded into C++
def create_libtorch_script(model, train_id, checkpoint_name='model_best'):
    sm = torch.jit.script(model)
    model_script_name = 'DNR-{}-{}_model.pt'.format(train_id, checkpoint_name)
    model_script_path = os.path.join(PATH, 'libtorch-models', model_script_name)
    sm.save(model_script_path)
    print(model_script_path)

In [ ]:
# Visualize a model prediction
def generate_images(model, test_input, tar):
    prediction = model(test_input)#, training=True)
    plt.figure(figsize=(20,20))
    
    _, h, w, c = test_input.shape
    test_input_color = torch.zeros((h, w, 3))#, dtype=type(test_input))
    test_input_color = test_input[:,:,:, 0]
    tar = tar.permute(0, 2, 3, 1)
    prediction = prediction.detach().permute(0, 2, 3, 1)

    display_list = [test_input_color[0].numpy(), tar[0].numpy(), prediction[0].numpy()]
    title = ['Input Image', 'Ground Truth', 'Predicted Image']

    for i in range(3):
        plt.subplot(1, 3, i+1)
        plt.title(title[i])
        # getting the pixel values between [0, 1] to plot it.
        plt.imshow(display_list[i] * 0.5 + 0.5)
        plt.axis('off')
    plt.show()

In [ ]:
def generate_comparison(display_images, title, title_color):
    for i, image in enumerate(display_images):
        display_images[i] = image.permute(0, 2, 3, 1)
        
    # Should assert that rows * cols == len(title) == len(display_images)
    img_per_row = 2 # 3
    rows, cols = np.ceil(len(display_images) / img_per_row), np.min([len(display_images), img_per_row])
    plt.figure(figsize=(35 * cols,30 * rows))
    for i in range(len(display_images)):
        plt.subplot(rows, cols, i+1)
        plt.title(title[i], color=title_color[i])
        # getting the pixel values between [0, 1] to plot it.
        plt.imshow(display_images[i][0].numpy() * 0.5 + 0.5)
        #plt.axis('off')
    plt.show()

In [ ]:
def generate_mae(display_images, target, title):
    diffs = []
    target = target.permute(0, 2, 3, 1)
    for i, image in enumerate(display_images):
        display_images[i] = image.permute(0, 2, 3, 1)
        diff = torch.sum(torch.abs(target - display_images[i]), axis=3)
        diffs.append(diff)
        
    
    
    # Should assert that rows * cols == len(title) == len(display_images)
    rows, cols = len(display_images), 2
    plt.figure(figsize=(8,8))
    plt.subplot(1, 1, 1)
    plt.title("Ground truth")
    plt.imshow(target[0].numpy() * 0.5 + 0.5)
    plt.figure(figsize=(8 * cols,8 * rows))
    for i in range(0, len(display_images)):
        plt.subplot(rows, cols, 2*i+1)
        plt.title(title[i])
        # getting the pixel values between [0, 1] to plot it.
        plt.imshow(display_images[i][0].numpy() * 0.5 + 0.5)
        
        plt.subplot(rows, cols, 2*i+2)
        plt.title('MAE. Mean: {}'.format(torch.mean(diffs[i])))
        plt.imshow(1.0 - (diffs[i][0] / 2).numpy(), cmap='gray')
        #plt.axis('off')
    plt.show()

In [ ]:
def visualize_texture(neural_texture, select=None, disp_channels=None):
    mipmap = [neural_texture.mipmap_0,
              neural_texture.mipmap_1,
              neural_texture.mipmap_2,
              neural_texture.mipmap_3]
    
    _, _, _, size = mipmap[0].shape
    sample = 0
    for i, texture in enumerate(mipmap):
        if select is not None and i != select:
            continue
        sample += F.interpolate(texture.detach(), size=size, mode='bilinear', align_corners=False)
    
    sample = sample[0,:,:,:]
    sample = sample.permute(1, 2, 0)
    
    height, width, channels = sample.shape
    
    if disp_channels is None:
        disp_channels = channels
    
    plt.figure(figsize=(20, 20))
    for i in range(disp_channels):
        plt.title('Channel {}'.format(i))
        plt.subplot(np.ceil(channels / 4), 4, i+1)
        plt.imshow(sample[:, :, i].numpy() * 0.5 + 0.5)
    plt.show()

In [ ]:
def visualize_color_texture(neural_texture):
    mipmap = [neural_texture.mipmap_0,
              neural_texture.mipmap_1,
              neural_texture.mipmap_2,
              neural_texture.mipmap_3]
    
    _, _, _, size = mipmap[0].shape
    sample = 0
    for i, texture in enumerate(mipmap):
        sample += F.interpolate(texture[:,0:3,:,:].detach(), size=size, mode='bilinear', align_corners=False)
    
    sample = sample[0,:,:,:]
    sample = sample.permute(1, 2, 0)
    
    rescale = 2 / (torch.min(sample) - torch.max(sample))
    sample = sample * rescale
    print(torch.min(sample), torch.max(sample)) 
    
    plt.figure(figsize=(20, 20))
    plt.title('Color')
    #plt.subplot(1, 1, 1)
    plt.imshow(sample.numpy() * 0.5 + 0.5)
    plt.show()

In [ ]:
##-- Execute code below --##

In [ ]:
# List train ids here #
#train_ids = ['0626_000812', '0710_005202', '0714_232542', '0716_003726', '0716_152355', '0717_005959']
#train_ids = ['0714_232542', '0716_003726', '0716_152355', '0717_005959']
#titles = ['Single NT Layer, LowQ UV Coords, 1024', 'Single NT Layer, LowQ UV Coords, 1024, Stronger Data Aug', 'NT Hierarchies, LowQ UV Coords, 1024', 'NT Hierarchies, HighQ Coords, 1024', 'NT Hierarchies, HighQ UV Coords, 2048', 'Filtered Dataset', 'Colorized Dataset']
#train_ids = ['0710_005202', '0711_085844', '0714_232542', '0716_152355', '0717_005959', '0721_015234', '0722_010614']

titles = ['High-res Model', 'Low-res Model']
train_ids = ['0724_011558', '0724_122900']
uv_folder_names = ['uv/scene_highres_vertexuv_cubed_proj', 'uv/scene_lowres_vertexuv_cubed_proj']
models = load_trained_models(train_ids)

In [ ]:
def visualize_all_textures():
    num_layers = 4
    for train_id in train_ids:
        print('==================== \/ == Model', train_id, '== \/ ====================')
        visualize_color_texture(models[train_id].neural_texture)
        for layer in range(num_layers):
            print('--> Layer:', layer)
            visualize_texture(models[train_id].neural_texture, select=layer, disp_channels=4)

# Run
visualize_all_textures()

In [ ]:
# Show a single validation input, ground truth and preducted sample from the first model #
def display_uv_ground_truth_predicted(uv_folder_name, filter_file):
    loader = UVDataLoader('data/scene0000_00', uv_folder_name, 'color', filter_file, 1, shuffle=False, skip=6, compressed_input=True).split_validation()
    for batch_idx, (data, target) in enumerate(loader):
        for train_id in train_ids:
            print('Train ID:', train_id)
            model = models[train_id]  
            generate_images(model, data, target)
        break

# Run over all models for a specific dataset
display_uv_ground_truth_predicted(uv_folder_names[0], filter_file='filters/keypoint_blur_1.2.txt')

In [ ]:
# Show a validation sample prediction for each model #
def display_prediction_mae(frame_index, filter_file, size):
    display_images = []
    for i, train_id in enumerate(train_ids):
        # Load from the validatiom dataset
        loader = UVDataLoader('data', uv_folder_names[i], 'color', filter_file, 1, shuffle=False, skip=6, compressed_input=True, size=size)
        for batch_idx, (data, target) in enumerate(loader):
            if batch_idx < frame_index:
                continue

            # Get the trained model
            model = models[train_id]

            # Make a prediction using the model
            prediction = model(data)
            prediction = prediction.detach()
            display_images.append(prediction)
            break

    # Plot results
    generate_mae(display_images, target.detach(), titles)

# Run
display_prediction_mae(frame_index=20, filter_file='filters/keypoint_blur_1.2.txt', size=(968, 1296)):

In [ ]:
# Show a validation sample prediction for each model #

display_images = []
# Load from the validatiom dataset
loader = UVDataLoader('data', 'data/keep_1.2.txt', 1, True, 6).split_validation()
for batch_idx, (data, target) in enumerate(loader):
    print('data size:', data.size)
    # Add target image used to generate predictions
    display_images.append(target)

    # Add predictions
    for train_id in train_ids:
        # Get the trained model
        model = models[train_id]
        
        # Make a prediction using the model
        prediction = model(data)
        prediction = prediction.detach()
        display_images.append(prediction)
    break

# Plot results
title = ['Ground Truth', 'Prediction Exp 1: 521 train, 104 val',
         'Prediction Exp 1: 1042 train, 208 val',
         'Prediction Exp 1: 2083 train, 417 val', '', '', '']
title_color = ['black', 'magenta', 'green', 'blue', 'blue', 'blue', 'blue']

generate_comparison(display_images, title, title_color)

In [ ]:
# Create a libtorch script file from model #
def generate_torchscript_model(train_id)
    print(train_id)
    script_model = load_trained_model(train_id)
    create_libtorch_script(script_model, train_id)
    
# Run
generate_torchscript_model(train_id='0717_005959')